In [1]:
import sys
import dash
import plotly
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
from datetime import datetime
import locale

import warnings
warnings.simplefilter("ignore")

print('Python: {}'.format(sys.version))
print('Dash: {}'.format(dash.__version__))
print('Plotly: {}'.format(plotly.__version__))
print('Pandas: {}'.format(pd.__version__))

/tmp/ipykernel_117231/3715833163.py:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/tmp/ipykernel_117231/3715833163.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


Python: 3.11.6 (main, Oct  3 2023, 00:00:00) [GCC 13.2.1 20230728 (Red Hat 13.2.1-1)]
Dash: 2.14.2
Plotly: 5.11.0
Pandas: 1.5.2


### Load data

In [2]:
file = '/home/darlan/Documents/RAF.csv'
df = pd.read_csv(file, sep =',')

In [4]:
print(df.shape)

(846, 25)


In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 846 entries, 0 to 845
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   mes                        846 non-null    object 
 1   ano                        638 non-null    float64
 2   ocorrencia                 817 non-null    object 
 3   uc                         828 non-null    object 
 4   pf_pj                      828 non-null    object 
 5   eh_cliente                 828 non-null    object 
 6   nome_cliente               846 non-null    object 
 7   motivo_reclamacao          824 non-null    object 
 8   sub_motivo_sys             827 non-null    object 
 9   reclamacao_precede         827 non-null    object 
 10  ja_reclamou                839 non-null    object 
 11  cidade                     598 non-null    object 
 12  estado                     830 non-null    object 
 13  id_cms                     587 non-null    object 

In [84]:
#df = df.drop('quem_ligou', axis=1)
df = df.drop('mes_referencia', axis=1)
df = df.drop('motivos_contato', axis=1)

df = df[df['voltaria_a_fazer_negocios'].notna()]
df = df[df['id_ra'].notna()]
df = df[df['id_cms'].notna()]
df = df[df['pf_pj'].notna()]
df = df[df['ocorrencia'].notna()]
df = df[df['respondida'].notna()]
df = df[df['reclamacao_precede'].notna()]
df = df[df['sub_motivo_sys'].notna()]

df = df[df['eh_cliente'] != '-']
df = df[df['reclamacao_precede'] != '-']
df = df[df['cidade'] != '0']
df = df[df['uc'] != '0']

df['voz_cliente'].fillna('Dor', inplace = True)
df['quem_ligou'].fillna('Não identificado', inplace = True)
df['motivo_reclamacao'].fillna('Cobrança Indevida', inplace = True)

df['ano'].fillna(df['ocorrencia'].str[:4], inplace = True)
df['cidade'].fillna('Belo Horizonte', inplace = True)

df['pf_pj'] = df['pf_pj'].replace('-', 'PF')
df['pf_pj'] = df['pf_pj'].replace('0', 'PF')
df['pf_pj'] = df['pf_pj'].str.upper()

df['eh_cliente'] = df['eh_cliente'].str.upper()
df['eh_cliente'] = df['eh_cliente'].replace('SIIM', 'SIM')
df['eh_cliente'] = df['eh_cliente'].replace('NÃO LOCALIZADO', 'NÃO')
df['eh_cliente'] = df['eh_cliente'].replace('0', 'NÃO')

df['voz_cliente'] = df['voz_cliente'].str.upper()
df['voz_cliente'] = df['voz_cliente'].replace('RECLAME AQUI', 'DOR')

df['reclamacao_precede'] = df['reclamacao_precede'].str.upper()

df['respondida'] = df['respondida'].str.upper()
df['respondida'] = df['respondida'].replace('SM', 'SIM')
df['respondida'] = df['respondida'].replace('SIM - LIGAÇÃO', 'SIM')
df['respondida'] = df['respondida'].replace('SIM - WHATSAPP', 'SIM')
df['respondida'] = df['respondida'].replace('DESATIVADA NO RA', 'NÃO')
df['respondida'] = df['respondida'].replace('DESATIVADA NO RA', 'NÃO')
df['respondida'] = df['respondida'].replace('AINDA NÃO', 'SIM')

df['voltaria_a_fazer_negocios'] = df['voltaria_a_fazer_negocios'].str.upper()

df['data_resposta'] = df['data_resposta'].replace('06/04/203', '2023/04/06')
locale.setlocale(locale.LC_ALL, 'pt_BR')

formato = "%Y/%m/%d"

df['data_abertura'] = df['data_abertura'].apply(lambda linha: datetime.strptime(linha, formato))
df['data_resposta'] = df['data_resposta'].apply(lambda linha: datetime.strptime(linha, formato) if pd.notnull(linha) and linha != '_' else pd.NA)


df['nota'] = df['nota'].replace('-', None)
df['nota'] = df['nota'].apply(lambda number: int(number) if pd.notnull(number) else pd.NA)


df['ano'] = df['ano'].replace(2023.0, 2023)
df['ano'] = df['ano'].replace('2023', 2023)

In [85]:
months = {"Janeiro":1, "Fevereiro":2, "Março":3, "Abril":4, "Maio":5, "Junho":6, 
          "Julho":7, "Agosto":8, "Setembro":9, "Outubro":10, "Novembro":11, "Dezembro":12}

df['mes_num'] = df['mes'].map(months)

In [86]:
def apply_nps_desc(number):
    if number is not None and type(number) == int:
        if number in (9,10):
            return 'PROMOTOR'
        if number in (7,8):
            return 'PASSIVO'
        if number < 7:
            return 'DETRATOR'
    
    return ''

In [87]:
df['nota'] = df['nota'].apply(lambda number: int(number) if pd.notnull(number) else pd.NA)


In [88]:
df['nps_desc'] = df['nota'].apply(apply_nps_desc)

In [89]:
df.to_csv('data_complaints.csv', sep=',', index=False, encoding='utf-8')